# Simple Search Engine
Elasticsearch와 mysql을 결합한 검색엔진을 만들어보겠습니다.
____

In [ ]:
import pandas as pd
df = pd.read_csv("movie_doc.csv",sep="\t")

In [ ]:
df.head()

## 2. MySQL에 데이터 적재

In [ ]:
import mysql.connector


HOST = "localhost"
USER = "root"
PASSWORD = "123456789"
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database="movie_db"
)

mycursor = mydb.cursor()


In [ ]:
sql = "SELECT * FROM movie"
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)

## 3. ElasticSearch에 데이터 인덱싱

In [ ]:
from elasticsearch import Elasticsearch, helpers


In [ ]:
import pprint  
INDEX_NAME = "movie_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer"
          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "story" : {
          "type" : "text",
          "analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean"
        }
      }

  }
}


In [ ]:


try:
    es.transport.close()
except:
    pass
es = Elasticsearch()



In [ ]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

In [ ]:
movies = [
    {
        "_index": INDEX_NAME,
        "_id" : doc['id'],
        "_source": {
            "title": doc['title'],
            "story": doc['story'],
        }
    }
    for doc in myresult
]

movies[:10]

In [ ]:
try:
    response = helpers.bulk(es, movies)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)

In [ ]:
doc = es.get(index=INDEX_NAME, id=1)
pprint.pprint(doc)

## 4. ElasticSearch에서 검색후, 검색결과를 MySQL에서 재 검색

In [ ]:
query="은퇴한 킬러"
res = es.search(index=INDEX_NAME, q=query)
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))
    print("Title: {}".format(hit['_source']['title']))
    print("줄거리: {}".format(hit['_source']['story']))


In [ ]:
movie_id = res['hits']['hits'][0]["_id"]

In [ ]:
sql = "SELECT * FROM movie WHERE id = {}".format(movie_id)
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)